# Diferenciální rovnice 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp

## Exponenciální růst

Namodelujeme exponenciální růst 
$$\frac{\mathrm dx}{\mathrm dt}=kx$$
pro tři různé parametry $k$. Protože po transformaci do bezrozměrného času $\tau = kt$ má rovnice tvar 
$$\frac{\mathrm dx}{\mathrm d\tau }=x$$
nezávislý na $k$, ověříme, že všechna řešení po transformaci do bezrozměrného času splývají.


Nejprve růst pro tři různé hodnoty koeficientu $k$. Vidíme, že čím vyšší je $k$, tím rychleji velikost populace roste v čase.

In [ ]:
### Příprava funkcí a parametrů
pocatecni_podminka = np.array([0.1])  # počáteční podmínka
meze = np.array([0,10]) # interval, na kterém hledáme řešení
parametry = [1,1.5,2] # seznam parametrů
n = 100
def rovnice(t, x, k=1):
    return k*x

### Řešení modelu
t=np.linspace(*meze, n)  # definiční obor, v těchto bodech budeme hledat řešení
df = pd.DataFrame()      # tabulka pro výstup
df.index = t              # sloupec s časem

for parametr in parametry:
    reseni = solve_ivp(
                       lambda t,x:rovnice(t,x,k=parametr),
                       meze,
                       pocatecni_podminka,
                       t_eval=t
                       )
    df[parametr] = reseni.y.T # další sloupec tabulky
    # lambda funkce viz https://www.w3schools.com/python/python_lambda.asp
    # (dočasná nepojmenovaná funkce)

### Vizualizace řešení
ax = df.plot()
ax.set(
    ylim = (0,1000),
    title = "Model exponenciálního růstu",
    xlabel="čas",
    ylabel="velikost populace",
)
plt.legend(title="Hodnoty parametru")

Po transformaci do bezrozměrného času by měly křivky splynout. Abychom to viděli graficky, budeme postupně snižovat jejich tloušťku a nastavíme částečnou průhlednost. To zajistíme volbami `lw` (zkratka za `linewidth`) a `alpha`.

In [ ]:
for k,lw in zip(parametry,[8,5,2]):
    plt.plot(t*k,df[k],lw=lw,label=k,alpha=0.5)  # Na vodorovne ose neni cas ale bezrozmerny cas
ax = plt.gca()
ax.set(
    ylim=(0,100),
    xlim=(0,10),
    xlabel="bezrozměrný čas",
    ylabel="velikost populace",
    title="Růst rychlostí úměrnou velikosti v bezrozměrném čase"
)
plt.legend(title=r"Hodnota $k$");

## Exponenciální růst k vodorovné asymptotě

### Úkol

Nakreslete řešení rovnice růstu k vodorovné asymptotě (von Bertalanffyho růst)
$$\frac{\mathrm dL}{\mathrm dt}=k(L_\infty-L)$$
pro tři různé hodnoty parametru $L_\infty$.

In [ ]:
# Sem vložte řešení.

## Logistický růst

### Úkol

Vykreslete řešení logistické rovnice $$\frac{\mathrm dx}{\mathrm dt}=rx\left
(1-\frac xK\right)$$
pro cca deset počátečních podmínek rovnoměrně rozmístěných mezi $0$ a $K$.

In [ ]:
# Sem vložte řešení. Můžete jej rozdělit do více buněk, které si sem vložíte.

### Logistická rovnice v bezrozměrném tvaru 

Nyní si ukážeme, že v určitém smyslu stačí bez újmy na obecnosti studovat případ $K=1$ a $r=1$. Na rovnici 
$$\frac{\mathrm dy}{\mathrm d\tau}=y(1-y)$$
se totiž logistická rovnice transformuje při přechodu od rovnice 
$$\frac{\mathrm dx}{\mathrm d t}=rx\left(1-\frac xK\right)$$
s použitím substituce $y=\frac xK$ a $\tau = rt$. Najdeme $y(\tau)$ jako řešení bezrozměrné rovnice s jednotkovými parametry a $x(t)$ jako řešení rovnice s náhodnými parametry $r$ a $K$. Poté ukážeme na několika numerických hodnotách, že platí $$y(t) = K x(rt).$$




In [ ]:
pocatecni_podminka = np.array([0.1])
meze = np.array([0,10])
def rovnice(t, x, r=1, K=1):
    return r*x*(1-x/K)

# r a K budou náhodná čísla z intervalu (0,0.2) a (0,10).
K,r = np.array([10,0.2])*np.random.random(2)
print(f"r={r}, K={K}")
t = np.linspace(*meze,12)

reseni_plne_rovnice = solve_ivp(
                   lambda t,x:rovnice(t,x,r=r,K=K),
                   meze,
                   pocatecni_podminka,
                   t_eval=t
                   )

reseni_bezrozmerne_rovnice = solve_ivp(
                   rovnice,
                   r*meze,  # převod mezí na bezrozměrný čas
                   pocatecni_podminka/K,  # převod na bezrozměrnou velikost populace
                   t_eval=r*t # vyhodnoceni v bezrozmernem case
                   )

df = pd.DataFrame()
df.index = t
df["plnohodnotna"] = reseni_plne_rovnice.y[0]
bezrozmerna_velikost = reseni_bezrozmerne_rovnice.y[0] 
velikost_populace = K*bezrozmerna_velikost
df["bezrozmerna"] = velikost_populace
df["rozdil"] = np.abs(df["bezrozmerna"]-df["plnohodnotna"])
df["relativni rozdil"] = df["rozdil"]/df["plnohodnotna"]
df

In [ ]:
df.rozdil.max()

In [ ]:
df["relativni rozdil"].max()

In [ ]:
np.isclose(df["bezrozmerna"],df["plnohodnotna"], rtol=0.0001).all()